In [12]:
import pandas as pd
import numpy as np
import pydeck as pdk
import math

In [2]:
# Lightstation data
ls_data = pd.read_csv('./data/BC_Lightstations_and_other_Sample_Sites.csv', skiprows=1)
ls_data.rename(columns={
    'LATITUDE (DECIMAL DEGREES)': 'latitude',
    'Unnamed: 4': 'longitude',
    'LIGHTSTATION / LOCATION': 'lightstation'
}, inplace=True)
ls_data.rename(str.lower, axis='columns', inplace=True)
ls_active = ls_data[ls_data['data collection status']=='ACTIVE']
ls_inactive = ls_data[ls_data['data collection status']!='ACTIVE']


In [5]:
# Fish farm data
ff_data = pd.read_csv('./data/lice-count-dens-pou-2011-ongoing-rpt-pac-dfo-mpo-aquaculture-eng.csv')
ff_data.columns
ff_data.rename(columns = {
    'Latitude (Decimal Degrees)': 'latitude',
    'Longitude (Decimal Degrees)': 'longitude',
    'Facility Reference \nNumber': 'facility'
}, inplace=True)
ff_data.rename(str.lower, axis='columns', inplace=True)


In [83]:
ff_sites = ff_data[['facility', 'latitude', 'longitude']]
ff_sites = ff_sites.drop_duplicates(subset=['facility'])

In [7]:
midpoint = (np.median(ff_sites["latitude"]), np.median(ff_sites["longitude"]))

farm_colours = [[240,249,232],[204,235,197],[168,221,181],[123,204,196],[67,162,202],[8,104,172]]
lightstation_colours = [(254,240,217),(253,212,158),(253,187,132),(252,141,89),(227,74,51),(179,0,0)]
map = pdk.Deck(
    initial_view_state={
        "latitude": midpoint[0],
        "longitude": midpoint[1],
        "zoom": 6,
        "pitch": 50,
    },
    layers=[
        pdk.Layer(
        "HeatmapLayer",
        ff_sites,
        get_position=['longitude', 'latitude'],
        opacity=0.5,
        color_range=farm_colours
        ),
        pdk.Layer(
        "HeatmapLayer",
        ls_active,
        get_position=['longitude', 'latitude'],
        opacity=0.5,
        color_range=lightstation_colours
        ),
    ],
    mapbox_key='pk.eyJ1IjoidG9tc2l0dGVyIiwiYSI6ImNrYjJocjU0azBiNTEycnFwZWFkajB2bnoifQ.M1BDSPntGHfosotYAUAINw'
)

print("Farms are in blue and lightstation locations are in red")
map.to_html('farms.html')

Farms are in blue and lightstation locations are in red


'C:\\Users\\Tom\\Projects\\sea-lice\\farms.html'

In [84]:


#min_lat = math.floor(ff_sites['latitude']
print("Min:", ff_sites.min(axis='rows'))
print("Max:", ff_sites.max(axis='rows'))

Min: facility      78.00000
latitude      48.94310
longitude   -128.57494
dtype: float64
Max: facility     7714.00000
latitude       52.80048
longitude    -123.72455
dtype: float64


In [21]:
min_lat = math.floor(np.min(ff_sites.latitude))
min_lon = math.floor(np.min(ff_sites.longitude))

max_lat = math.ceil(np.max(ff_sites.latitude))
max_lon = math.ceil(np.max(ff_sites.longitude))

In [29]:
# % ./subset_dataset.py -s 20100101 -f 20100201 -b -140 -110 20 30 -x MUR-JPL-L4-GLOB-v4.1
# Subset the data from 1 Jan 2010 to 1 Feb 2010 in a box from -140 to -110 degrees longitude and 20 to 30 degrees latitude
# for shortName MUR-JPL-L4-GLOB-v4.1

!python ./nasa_subset_data.py -s 20190101 -b {min_lon} {max_lon} {min_lat} {max_lat} -x MUR-JPL-L4-GLOB-v4.1


Please wait while program searching for the granules ...



Traceback (most recent call last):
  File "./nasa_subset_data.py", line 542, in <module>
    standalone_main()
  File "./nasa_subset_data.py", line 267, in standalone_main
    doc = minidom.parse(urllib.request.urlopen(samplefile))
  File "C:\Users\Tom\AppData\Local\Programs\Python\Python37\lib\urllib\request.py", line 222, in urlopen
    return opener.open(url, data, timeout)
  File "C:\Users\Tom\AppData\Local\Programs\Python\Python37\lib\urllib\request.py", line 531, in open
    response = meth(req, response)
  File "C:\Users\Tom\AppData\Local\Programs\Python\Python37\lib\urllib\request.py", line 641, in http_response
    'http', request, response, code, msg, hdrs)
  File "C:\Users\Tom\AppData\Local\Programs\Python\Python37\lib\urllib\request.py", line 569, in error
    return self._call_chain(*args)
  File "C:\Users\Tom\AppData\Local\Programs\Python\Python37\lib\urllib\request.py", line 503, in _call_chain
    result = func(*args)
  File "C:\Users\Tom\AppData\Local\Programs\Python\P

In [35]:
sst = Dataset("./data/20190101090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1_subset.nc", "r", format="NETCDF4")

In [79]:
sst.variables['analysed_sst'].shape
sst_lats = sst.variables['lat'][:]
sst_lons = sst.variables['lon'][:]
temps = sst.variables['analysed_sst']

In [103]:
ff_sites = ff_sites.round({'latitude': 2, 'longitude': 2})

for index, site in ff_sites.iterrows():
    lat, lon = site.latitude, site.longitude
    lat_loc = np.abs(sst_lats - lat).argmin()
    lon_loc = np.abs(sst_lons - lon).argmin()
    temp = temps[0, lat_loc-2:lat_loc+2, lon_loc-2:lon_loc+2]
    print(f"Temp at ({lat}, {lon}) is {round(temp.mean(), 1)}")


Temp at (48.94, -124.99) is 281.4
Temp at (49.31, -126.01) is 281.9
Temp at (49.27, -125.81) is 281.9
Temp at (49.34, -125.95) is 281.9
Temp at (49.4, -126.15) is 281.8
Temp at (49.24, -125.75) is 281.9
Temp at (49.31, -125.82) is 281.9
Temp at (49.38, -126.09) is 281.8
Temp at (49.26, -125.87) is 282.0
Temp at (49.26, -125.84) is 282.0
Temp at (49.32, -126.05) is 281.9
Temp at (49.25, -125.91) is 282.0
Temp at (49.28, -125.83) is 281.9
Temp at (49.66, -126.45) is 281.6
Temp at (49.66, -126.48) is 281.7
Temp at (49.88, -126.76) is 281.6
Temp at (49.64, -126.34) is 281.6
Temp at (49.89, -126.79) is 281.6
Temp at (49.66, -126.43) is 281.6
Temp at (50.48, -127.73) is 281.7
Temp at (50.46, -127.89) is 281.9
Temp at (50.47, -127.79) is 281.8
Temp at (50.47, -127.84) is 281.8
Temp at (49.78, -124.15) is 280.5
Temp at (49.8, -124.1) is 280.4
Temp at (49.63, -123.84) is 280.5
Temp at (50.32, -125.26) is 280.4
Temp at (50.18, -125.15) is 280.5
Temp at (50.32, -125.01) is 280.4
Temp at (50.45, -

In [100]:
temp.mean()

281.4173333333333

In [71]:
def k_to_c(temp):
    return temp - 273.15